In [1]:
# ipython settings
%load_ext autoreload
%autoreload 2
import warnings

warnings.filterwarnings('ignore')

# 3. Synthetic experiments with Symbolic Pursuit

In this notebook, we shall reproduce one of the experiments from Section 6.1 of the paper.
The idea is to start with a linear pseudo black-box for which the importance vector is known unambiguously and see which interpretability methods identifies this vector the most precisely. Let us start by the useful imports.


In [2]:
from symbolic_pursuit.models import SymbolicRegressor  # our symbolic model class
from sklearn.metrics import mean_squared_error # we are going to assess the quality of the model based on the generalization MSE
from sympy import init_printing # We use sympy to display mathematical expresssions 
import numpy as np # we use numpy to deal with arrays
import lime 
import lime.lime_tabular
init_printing()

We now define a linear pseudo black-box $f$ defined on a 3 dimensional feature space.

$$ f(x_1,x_2,x_3)= x_1 + 2 \cdot x_2 + 3 \cdot x_3$$ 

The importance vector associated to this model is trivially given by $\beta = (1,2,3)$ In this case, we shall keep it unnormalised, unlike in the main paper as we deal with few examples. Let us translate this in Python. 

In [3]:
def f(X):
    return -1*X[:, 0]**2 + 2*X[:,1]**2 + 3*X[:,2]**3

dim_X = 3

Now draw uniformly 100 test points  that we will feed to a *LIME* explainer <cite data-cite="2480681/WCEBQ7N9"></cite> and to train a Symbolic model.

In [4]:
n_pts = 100
X = np.random.uniform(0, 1, (n_pts, dim_X))

Now we draw 10 test ponits $x_{test} \equiv U([0,1]^3)$ that we are going to use in order to evaluate the perfomances of both explainers on unseen data.

In [5]:
n_test = 10
X_test = np.random.uniform(0, 1, (n_test, dim_X))

Since LIME produces importance vectors with entries in the form $(feature \ domain , importance)$ for each feature appearing in decreasing order of importance, we implement a function which identifies the feature from the first entry of the tuple and who sorts the importances in the form $(importance(x_1), importance(x_2), importance(x_3))$.

In [6]:
def order_weights(exp_list):
    ordered_weights = [0 for _ in range(dim_X)]
    for tup in exp_list:
        feature_id = int(tup[0].split('x_')[1][0])
        ordered_weights[feature_id-1] = tup[1]    
    return ordered_weights    

We are now ready to extract the feature importance for our 10 test points as predicted by the LIME explainer :

In [7]:
lime_weight_list = []
explainer = lime.lime_tabular.LimeTabularExplainer(X, 
                                                   feature_names=["x_"+str(k) for k in range(1,dim_X+1)], 
                                                   class_names=['f'], 
                                                   verbose=True,
                                                   mode='regression')

for i in range(n_test):
    exp = explainer.explain_instance(X_test[i], f, num_features=dim_X)
    lime_weight_list.append(order_weights(exp.as_list()))  
                            
print(lime_weight_list)    

Intercept 1.2811686984317572
Prediction_local [0.48919162]
Right: 0.6759049269299239
Intercept 1.3673322185147874
Prediction_local [0.1313389]
Right: 0.04956741600730538
Intercept 0.9190793635741643
Prediction_local [1.51239224]
Right: 1.909123611002963
Intercept 0.9108872124177105
Prediction_local [1.5847062]
Right: 1.379830561639335
Intercept 0.6686939747994213
Prediction_local [2.27954227]
Right: 3.096074194835146
Intercept 0.8071440927164905
Prediction_local [1.91238276]
Right: 1.988105190628521
Intercept 0.851757766546742
Prediction_local [1.6822679]
Right: 1.8098883041410598
Intercept 1.2897077015445049
Prediction_local [0.40970882]
Right: 0.39518061870925936
Intercept 0.8213817009347409
Prediction_local [1.79123668]
Right: 1.5958066392171852
Intercept 1.6459679941564676
Prediction_local [-0.61403771]
Right: -0.7304062415318053
[[0.39494272822337434, -0.5564098017465704, -0.6305100041803118], [0.254529470126835, -0.8683692464852784, -0.6221535433809703], [-0.022640556660238752, -

As we can see from the last output, which is the list of predicted importance vectors, LIME seems to produce a big variety of importance vectors. This is suprising for a global linear model. We also note that the relative importance seem inconsistent with the true importance vector $\beta$ defined above. Let us now train a Symbolic model for $f$ based on our training set.

In [8]:
symbolic_model = SymbolicRegressor(maxiter=20,
                 eps=1.0e-4)
symbolic_model.fit(f, X)

Model created with the following hyperparameters :
 loss_tol=0.001 
 ratio_tol=0.9 
 maxiter=20 
 eps=0.0001 
 random_seed=42
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Now working on term number  1 .
Now working on hyperparameter tree number  1 .
         Current function value: 0.160665
         Iterations: 20
         Function evaluations: 328
         Gradient evaluations: 41
Now working on hyperparameter tree number  2 .
         Current function value: 0.135299
         Iterations: 20
         Function evaluations: 440
         Gradient evaluations: 44
Now working on hyperparameter tree number  3 .
         Current function value: 0.726129
         Iterations: 20
         Function evaluations: 583
         Gradient evaluations: 53
The tree number  2  was selected as the best.
Backfitting complete.
The current model has the following expression:  5.67923895476624*[ReLU(P1)]**1.21501288739573*hyper((-0.0194585592650769,), (1

We now ask our symbolic model to predict the importance vectors for each test point.

In [9]:
symbolic_weight_list = [] 
for k in range(n_test):
    symbolic_weight_list.append(symbolic_model.get_feature_importance(X_test[k]))
    

In [10]:
print(symbolic_weight_list)

[[-1.12402211668569, 1.92997336859991, 2.00904654050354], [-0.869135173565616, 1.02790109590833, 1.00504326432794], [-1.09658170149752, 1.28974856120211, 6.90008380751337], [-1.56127640927291, 3.15681347010351, 3.26971259371104], [-0.746895281437129, 2.39368610340778, 7.21156980281456], [-1.57189511237784, 2.95451158304893, 5.62198015843764], [-0.181731358705430, 1.61084204650646, 6.23821277600107], [-0.792015152141856, 1.34805007474474, 3.22781083510790], [-0.109356175623874, 1.64791521348413, 5.65982955836456], [-1.46171637131951, 0.169075959140978, -1.03272924226887]]


As we can see, our results appear to be always consistent and very close to the true importance vector $\beta$.

In [12]:
print (np.linalg.norm(symbolic_model.predict(X_test) - f(X_test)))

0.31590864325578144


In [14]:
symbolic_model.predict(X_test)

array([ 0.73712838,  0.0194304 ,  1.97291422,  1.38841182,  3.12368285,
        2.05240243,  1.75006513,  0.24550255,  1.41627369, -0.56340908])

In [15]:
f(X_test)

array([ 0.67590493,  0.04956742,  1.90912361,  1.37983056,  3.09607419,
        1.98810519,  1.8098883 ,  0.39518062,  1.59580664, -0.73040624])

## References<div class="cite2c-biblio"></div>

<div class="cite2c-biblio"></div>